# Plotting the price of assets against Elon Musk and Tesla Twitter activity 

## Why?

Over the last few months my news feed has been a constant wash of articles about Blockchain, stock markets (GME) and Elon Musk. The first and the last of these topics have been newsworthy regularly over the last few years, meanwhile the GME shortsqueeze appeared overnight.

There's no doubt that these topics are interconnected. Homebound US citizens with stimulus checks and too much time seem to be the main participants in these three hot topics.

Whilst this has been going on I have found myself increasingly interested in data science, more specifically data visualisation, I was introduced to the field by a colleague and sites like ChartR (https://www.chartr.co/) have gone a long way in furthering that interest. Stories told through a simple vis are compelling. A picture (or chart) speak a thousand words.

To attempt to get into the field I have been taking an extended course on data science provided by IBM. I found this course good, but lacking applied projects..so here goes an attempt at applying some of what I have known.

## Why this post?

This isn't really written to garner an audience - more for me to catalogue my progress over what I hope will be a long while.

## The project

The plan is to look at the Bitcoin, Dogecoin and GME prices along side Mr Musks Twitter activity to see if I can see any trends.

It should be noted this is not a rigerous quantitative approach - more a high level 'lets see what I can achieve in a day or so and check what I've learned' approach. If you're after the former then have a look at this: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3778844

In [175]:
#To start import pandas to enable easier data manipulation, Plotly for eventual visualisation, yfinance to obtain our data

import pandas as pd

import plotly.express as px

import yfinance as yf

import datetime as dt



In [177]:
# Download the required data from yahoo finance using the appropriate tickers 
# Using start Dec 2020 to current day
# Take daily prices on the assumption tweets are around that frequent



#Start by creating variables for the start date (user input) and the end date being today's date - to do that the datetime package is useful

today_date = dt.date.today()

today_date = today_date.strftime("%Y-%m-%d")

start_date = "2021-02-01"


#BTC
BTC_Data = yf.download(tickers='BTC-USD', start = start_date, end = today_date, interval = '1d')

#It's worth noting not reseting the index caused a major headache as the 'date' column header was misaligned without it.
BTC_Data.reset_index(inplace=True)

#DOGE
DOGE_Data = yf.download(tickers='DOGE-USD', start = start_date, end = today_date, interval = '1d')

DOGE_Data.reset_index(inplace=True)

#GME
GME_Data = yf.download(tickers='GME', start = start_date, end = today_date, interval = '1d')

GME_Data.reset_index(inplace=True)

#I tried running the head of all three downnloads but Python will only show one so that will suffice to show its working and what we have returned

DOGE_Data_df = pd.DataFrame(DOGE_Data)

DOGE_Data_df['Date'] = pd.to_datetime(DOGE_Data_df['Date']).dt.date


BTC_Data_df = pd.DataFrame(BTC_Data)

BTC_Data_df['Date'] = pd.to_datetime(BTC_Data_df['Date']).dt.date

GME_Data_df = pd.DataFrame(GME_Data)

GME_Data_df['Date'] = pd.to_datetime(GME_Data_df['Date']).dt.date


GME_Data_df.tail(5)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
28,2021-03-12,275.000000,295.500000,262.269989,264.500000,264.500000,25760700
29,2021-03-15,277.519989,283.000000,206.000000,220.139999,220.139999,24100400
30,2021-03-16,203.160004,220.699997,172.350006,208.169998,208.169998,35241900
31,2021-03-17,217.839996,231.470001,204.000000,209.809998,209.809998,16259300
32,2021-03-18,214.000000,218.880005,195.649994,201.750000,201.750000,11764900


### Quick note on obtaining tweets

How would we obtain the tweet information? My initial thought process was:

- Scrape the tweets using the Twitter API 
- Create a table of the tweets with datestamp and text
- Filter by keyword e.g. BTC, Bitcoin etc.
- Plot the tweets using date stamp over the price graph 

this would likely involve appending prices to the table so that we can plot an effective 'x and y' coordinate of date and price and then use the text as a data label - that's the theory at least...

In scraping the tweets I came across Tweepy, which is a convinient pre-made package that enables you to scrape tweets without too much prior knowledge. I ended up using it and an article from Martin Beck: https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1 to achieve what I wanted. I won't claim I spent too much time examining what was going on in the next cell but I cared more that it worked, and really that's the beauty of the ds community - open knowledge sharing. It's what makes it approachable to a complete novice like myself. The cell below is far better explained by the article above so I won't go into much detail.



In [155]:
# Import what is required
import tweepy
import time

# Credentials to enable access to the Twitter API

consumer_key = "qSlXdQBHiP3bd45cCOs9Sq0FO"
consumer_secret = "I3W8CBFDkhgwCFp6VExQj3FhTHEkcuMXMAlgrj88MS7WaEQxP3"
access_token = "604056369-pGIpZHnepCzzs4yXdF9clWV10UxLy1mX7kEhW4ir"
access_token_secret = "0pHtZQgE2QHc4tP9JyU5VCKuTjjHdh38Qxzpz8MUlWZex"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

#Here the code defines a function that we can call later on to get the data we need

tweets = []

def username_tweets_to_csv(username,count):
    try:      
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]

        # Creation of dataframe from tweets list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list,columns=['Datetime', 'Tweet Id', 'Text'])

        # Converting dataframe to CSV 
        tweets_df.to_csv('{}-tweets.csv'.format(username), sep=',', index = False)

    except BaseException as e:
          print('failed on_status,',str(e))
          time.sleep(3)
        
# Input username to scrape tweets and name csv file
# Max recent tweets pulls x amount of most recent tweets from that user
username = 'elonmusk'
count = 250

# Calling function to turn username's past X amount of tweets into a CSV file
username_tweets_to_csv(username, count)

In [191]:
#import the created CSV files and create a pandas data frame

csv_path = '/Users/jamesdawson/Desktop/Data analysis /Python/elonmusk-tweets.csv'

df = pd.read_csv(csv_path)

df.head(5)

,Datetime,Tweet Id,Text
0,2021-03-19 08:25:36,1372826575293583366,@MarcusHouse The [ahem] reach-around
1,2021-03-19 07:32:18,1372813161846439936,@RationalEtienne @jordanbpeterson We could tal...
2,2021-03-19 07:28:55,1372812312936128513,@VinterNix @Bell Coming soon!
3,2021-03-19 07:12:22,1372808148369641476,@MoMoButFaster 🤣🤣
4,2021-03-19 06:54:27,1372803636305137664,"@remouherek @SciGuySpace As you can see, I hav..."


In [192]:
#Filter out anything that we don't feel is relevant from the tweets
#We need to create three data frames here so we can plot each respective one against a different plot of price

df['Datetime'] = pd.to_datetime(df['Datetime']).dt.date


In [193]:
#append the price to the line based on datetime - How easy this would be if I was using index match in excel..


df['BTC_price'] = df.Datetime.map(BTC_Data_df.set_index('Date')['Open'].to_dict())

df['DOGE_price'] = df.Datetime.map(DOGE_Data_df.set_index('Date')['Open'].to_dict())

df['GME_price'] = df.Datetime.map(GME_Data_df.set_index('Date')['Open'].to_dict())

df

,Datetime,Tweet Id,Text,BTC_price,DOGE_price,GME_price
0,2021-03-19,1372826575293583366,@MarcusHouse The [ahem] reach-around,57699.785156,0.057909,NaN
1,2021-03-19,1372813161846439936,@RationalEtienne @jordanbpeterson We could tal...,57699.785156,0.057909,NaN
2,2021-03-19,1372812312936128513,@VinterNix @Bell Coming soon!,57699.785156,0.057909,NaN
3,2021-03-19,1372808148369641476,@MoMoButFaster 🤣🤣,57699.785156,0.057909,NaN
4,2021-03-19,1372803636305137664,"@remouherek @SciGuySpace As you can see, I hav...",57699.785156,0.057909,NaN
5,2021-03-19,1372792458128412675,@jordanbpeterson What would you like to talk a...,57699.785156,0.057909,NaN
6,2021-03-19,1372792170147553292,@teslaownersSV 💯,57699.785156,0.057909,NaN
7,2021-03-19,1372739631846936581,https://t.co/Ngk8SJvDUW,57699.785156,0.057909,NaN
8,2021-03-19,1372725108909957121,@melaniemadri @thejackbeyer @NASASpaceflight F...,57699.785156,0.057909,NaN
9,2021-03-19,1372724555047972869,@thejackbeyer @NASASpaceflight The high bay is...,57699.785156,0.057909,NaN


### Comment on data collection

Though it would be nice to press run and we get everything we need pulled down without too much manual entry, at the end of the day you need consider whether it's worth it to automate some things. There's certain bits of news that I have in the back of my mind that aren't tweets but I do feel would compliment the analysis - to do this you can create manual annotations and place them onto the graph by eye.

In [194]:
df = df[df['Text'].str.contains("Haha|NASA")]


df


,Datetime,Tweet Id,Text,BTC_price,DOGE_price,GME_price
8,2021-03-19,1372725108909957121,@melaniemadri @thejackbeyer @NASASpaceflight F...,57699.785156,0.057909,NaN
9,2021-03-19,1372724555047972869,@thejackbeyer @NASASpaceflight The high bay is...,57699.785156,0.057909,NaN
14,2021-03-18,1372658174319337472,@lexfridman Haha,58893.078125,0.057640,214.0


In [195]:

fig_BTC = px.line(BTC_Data_df, x="Date", y="Open")

fig2_BTC = px.scatter(df, x="Datetime", y="BTC_price", text = "Text")

fig_BTC.add_trace(fig2_BTC.data[0])




fig_DOGE = px.line(DOGE_Data_df, x="Date", y="Open")

fig2_DOGE = px.scatter(df, x="Datetime", y="DOGE_price", text = "Text")

fig_DOGE.add_trace(fig2_DOGE.data[0])



fig_GME = px.line(GME_Data_df, x="Date", y="Open")

fig2_GME = px.scatter(df, x="Datetime", y="GME_price", text = "Text")

fig_GME.add_trace(fig2_GME.data[0])



fig_BTC.show()

fig_DOGE.show()

fig_GME.show()


#This hit the news straight away
#fig.add_annotation(x= "2021-01-29 00:00:00.00000", y=34111,
         #   text="Elon Musk changes his bio to #bitcoin",
         #   showarrow=True,
        #    arrowhead=1)


#A friend showed me this 
#fig.add_annotation(x= "2021-02-08 10:00:00.00000", y=46196,
        #    text="Tesla's 10-K return shows $1.5bn of bitcoin holdings",
        #    showarrow=True,
         #   arrowhead=1)

#As a holder of eth I remember this one quite well..
#fig.add_annotation(x= "2021-02-20 00:00:00.00000", y=55800,
     #       text="Elon Musk tweets 'BTC & ETH do seem high lol'",
     #       showarrow=True,
      #      arrowhead=1)


